In [3]:
import os
import json
import matplotlib.pyplot as plt
import networkx as nx

with open('Youtube_combined_Final.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

submissions = data

In [5]:
import networkx as nx
replyGraph = nx.DiGraph()
dSubCommentId = dict()

In [7]:
import json
import networkx as nx

# Load JSON
with open('Youtube_combined_Final.json', 'r', encoding='utf-8') as f:
    submissions = json.load(f)

replyGraph = nx.DiGraph()
dSubCommentId = {}

for vid in submissions:
    comments = vid.get('Comments', [])
    
    for comment in comments:
        author = comment.get('author')
        if author and author != '':
            if author in replyGraph:
                replyGraph.nodes[author]['subNum'] = replyGraph.nodes[author].get('subNum', 0) + 1
            else:
                replyGraph.add_node(author, subNum=1)

        # Map comment ID to author
        comment_id = comment.get('id')
        if comment_id and author:
            dSubCommentId[comment_id] = author

    # Now process replies
    for comment in comments:
        parent_author = comment.get('author')
        replies = comment.get('replies', [])
        for reply in replies:
            reply_author = reply.get('author')
            if reply_author and parent_author and reply_author != parent_author:
                # Ensure both nodes exist
                if reply_author not in replyGraph:
                    replyGraph.add_node(reply_author, subNum=0)
                if parent_author not in replyGraph:
                    replyGraph.add_node(parent_author, subNum=0)

                # Add or update edge
                if replyGraph.has_edge(reply_author, parent_author):
                    replyGraph[reply_author][parent_author]['replyNum'] += 1
                else:
                    replyGraph.add_edge(reply_author, parent_author, replyNum=1)

# Save the graph
nx.write_graphml(replyGraph, "youtube_reply_graph.graphml")
print("Graph saved as 'youtube_reply_graph.graphml'")


Graph saved as 'youtube_reply_graph.graphml'


In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

# Assuming 'graph' is your built DiGraph from before

plt.figure(figsize=(12, 12))

# Use a layout for better visualization
pos = nx.spring_layout(graph, k=0.15, iterations=20)

# Draw nodes sized by number of submissions (subNum)
node_sizes = [500 + 500*graph.nodes[n]['subNum'] for n in graph.nodes]

# Draw the graph
nx.draw_networkx_nodes(graph, pos, node_size=node_sizes, node_color='skyblue', alpha=0.7)
nx.draw_networkx_edges(graph, pos, arrowstyle='-|>', arrowsize=15, edge_color='gray')

# Draw node labels (authors)
nx.draw_networkx_labels(graph, pos, font_size=8)

plt.title("YouTube Comment Reply Graph")
plt.axis('off')
plt.show()


# Community Dectection

In [9]:
sFilename = 'youtube_reply_graph.graphml'
replyGraph = nx.readwrite.read_graphml(sFilename)

In [11]:
import community.community_louvain as community_louvain

G_undirected = nx.to_undirected(replyGraph)
partition = community_louvain.best_partition(G_undirected)

# Convert to community sets
def dictToSetFormat(comm_dict):
    comm_map = {}
    for node, group in comm_dict.items():
        comm_map.setdefault(group, set()).add(node)
    return list(comm_map.values())

communities = dictToSetFormat(partition)
print(f"Louvain detected {len(communities)} communities.")


Louvain detected 53632 communities.


In [ ]:
nx.draw(G_undirected, with_labels = True)

len(G_undirected.nodes), len(G_undirected.edges)